<a href="https://colab.research.google.com/github/BenardKemp/6SigmaMindV3/blob/main/Train001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q "trl==0.25.1" "transformers>=4.44.0" "accelerate>=0.34.2" \
"peft>=0.12.0" bitsandbytes datasets
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 41.0 MB/s eta 0:00:00


In [4]:
import trl, transformers, peft
print("TRL:", trl.__version__)
print("Transformers:", transformers.__version__)
print("PEFT:", peft.__version__)



TRL: 0.25.1
Transformers: 4.57.2
PEFT: 0.18.0


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "ibm-granite/granite-4.0-h-350m"  # 👉 change to your Granite variant

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,  # fp16 is safe on Colab GPUs
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

model.config.use_cache = False


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/681M [00:00<?, ?B/s]

The fast path is not available because one of `(selective_state_update, causal_conv1d_fn, causal_conv1d_update)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/excel_slm_dataset_complete00.jsonl"
)["train"]

len(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

86000

In [9]:
def format_example(example):
    instr = example["instruction"]
    out = example["output"].strip()

    example["text"] = (
        "You are an assistant that converts natural language into Excel formulas.\n\n"
        f"Instruction: {instr}\n"
        f"Excel formula: {out}"
    )
    return example

dataset = dataset.map(format_example)

Map:   0%|          | 0/86000 [00:00<?, ? examples/s]

In [10]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # may need tweaking per Granite architecture
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


In [20]:
from trl import SFTConfig

sft_config = SFTConfig(
    output_dir="/content/drive/MyDrive/granite-excel-lora",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=1000,
    fp16=True,          # use fp16
    bf16=False,
    max_length=512,
    packing=True,       # pack multiple samples per sequence
    gradient_checkpointing=True,
    report_to="none",
)

In [21]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,     # <-- NEW NAME in recent TRL
    train_dataset=dataset,
    peft_config=lora_config,
    args=sft_config,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [22]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,0.542900
100,0.285100
150,0.240800
200,0.218000
250,0.198000
300,0.182400
350,0.175800
400,0.171700
450,0.171700
500,0.168700


TrainOutput(global_step=522, training_loss=0.2325818461019874, metrics={'train_runtime': 9353.0399, 'train_samples_per_second': 0.892, 'train_steps_per_second': 0.056, 'total_flos': 6412324876519680.0, 'train_loss': 0.2325818461019874, 'entropy': 0.32860982811346084, 'num_tokens': 4057010.0, 'mean_token_accuracy': 0.9469070109454069, 'epoch': 1.0})

In [23]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=trainer.model,
    tokenizer=tokenizer,
    max_new_tokens=60,
    device_map="auto",
)

prompt = (
    "You are an assistant that converts natural language into Excel formulas.\n\n"
    "Instruction: Count how many cells in A2:A100 equal 'Error'.\n"
    "Excel formula:"
)

print(pipe(prompt)[0]["generated_text"])

Device set to use cuda:0
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in GraniteMoeHybridDecoderLayer. Setting `past_key_values=None`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: Input type (float) and bias type (c10::Half) should be the same

In [24]:
import torch

model = trainer.model  # your fine-tuned PEFT+Granite model

model.eval()
if hasattr(model, "gradient_checkpointing_disable"):
    model.gradient_checkpointing_disable()

model.config.use_cache = True  # safe & faster for inference


In [28]:
prompt = (
    "You are an assistant that converts natural language into Excel formulas.\n\n"
    "Instruction: Count how many cells in A2:A100 equal 'Error'.\n"
    "Excel formula:"
)

device = "cuda" if torch.cuda.is_available() else "cpu"

inputs_batch_encoding = tokenizer(
    prompt,
    return_tensors="pt"
)

# Move tensors to device and then cast to float16
inputs = {k: v.to(device).to(torch.float16) for k, v in inputs_batch_encoding.items()}


with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=60,
        do_sample=False,     # deterministic, good for formulas
        num_beams=1,
        use_cache=True,
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.HalfTensor instead (while checking arguments for embedding)

In [29]:
import torch

model = trainer.model  # your fine-tuned PEFT+Granite model

model.eval()
if hasattr(model, "gradient_checkpointing_disable"):
    model.gradient_checkpointing_disable()

model.config.use_cache = True  # safe & faster for inference


In [30]:
prompt = (
    "You are an assistant that converts natural language into Excel formulas.\n\n"
    "Instruction: Count how many cells in A2:A100 equal 'Error'.\n"
    "Excel formula:"
)

device = "cuda" if torch.cuda.is_available() else "cpu"

inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to(device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=60,
        do_sample=False,     # deterministic, good for formulas
        num_beams=1,
        use_cache=True,
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


RuntimeError: Input type (float) and bias type (c10::Half) should be the same

In [31]:
import torch

model = trainer.model
model.eval()

# make sure checkpointing is off and caching is on for inference
if hasattr(model, "gradient_checkpointing_disable"):
    model.gradient_checkpointing_disable()
model.config.use_cache = True

prompt = (
    "You are an assistant that converts natural language into Excel formulas.\n\n"
    "Instruction: Count how many cells in A2:A100 equal 'Error'.\n"
    "Excel formula:"
)

device = "cuda" if torch.cuda.is_available() else "cpu"

inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to(device)

with torch.inference_mode():
    # autocast to fp16 on CUDA so inputs/bias match
    if device == "cuda":
        with torch.autocast(device_type="cuda", dtype=torch.float16):
            output_ids = model.generate(
                **inputs,
                max_new_tokens=60,
                do_sample=False,   # deterministic for formulas
                num_beams=1,
                use_cache=True,
            )
    else:
        # CPU fallback, no autocast needed
        output_ids = model.generate(
            **inputs,
            max_new_tokens=60,
            do_sample=False,
            num_beams=1,
            use_cache=True,
        )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


You are an assistant that converts natural language into Excel formulas.

Instruction: Count how many cells in A2:A100 equal 'Error'.
Excel formula: =COUNTIF(A2:A100,"Error")


In [32]:
from huggingface_hub import notebook_login
notebook_login()

In [33]:
save_dir = "/content/drive/MyDrive/granite-excel-lora-2"

trainer.model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('/content/drive/MyDrive/granite-excel-lora-2/tokenizer_config.json',
 '/content/drive/MyDrive/granite-excel-lora-2/special_tokens_map.json',
 '/content/drive/MyDrive/granite-excel-lora-2/chat_template.jinja',
 '/content/drive/MyDrive/granite-excel-lora-2/vocab.json',
 '/content/drive/MyDrive/granite-excel-lora-2/merges.txt',
 '/content/drive/MyDrive/granite-excel-lora-2/added_tokens.json',
 '/content/drive/MyDrive/granite-excel-lora-2/tokenizer.json')

In [34]:
from huggingface_hub import create_repo

repo_id = "benkemp/granite-excel-formula-qlora"  # <- change this

# Create repo on HF (set private=True if you want)
create_repo(repo_id, private=False, exist_ok=True)

RepoUrl('https://huggingface.co/benkemp/granite-excel-formula-qlora', endpoint='https://huggingface.co', repo_type='model', repo_id='benkemp/granite-excel-formula-qlora')

In [35]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id=repo_id,
    folder_path=save_dir,
    commit_message="Initial upload of Granite Excel QLoRA model"
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:  84%|########3 |  550kB /  657kB            

CommitInfo(commit_url='https://huggingface.co/benkemp/granite-excel-formula-qlora/commit/e4643e855685721a45f6b8f6f5f3350698d7b106', commit_message='Initial upload of Granite Excel QLoRA model', commit_description='', oid='e4643e855685721a45f6b8f6f5f3350698d7b106', pr_url=None, repo_url=RepoUrl('https://huggingface.co/benkemp/granite-excel-formula-qlora', endpoint='https://huggingface.co', repo_type='model', repo_id='benkemp/granite-excel-formula-qlora'), pr_revision=None, pr_num=None)